In [1]:
# compile structure for irs soi data
# https://www.irs.gov/statistics/soi-tax-stats-historic-table-2

import re
import requests
import requests_cache
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

requests_cache.install_cache()

result = requests.get('https://www.irs.gov/statistics/soi-tax-stats-historic-table-2')
assert result.status_code == 200
c = result.content

soup = BeautifulSoup(c)
state_links = soup.find('table').find_all('a')

CACHED_DFS = {}

state_refs = []

for a in state_links:
    state_dict = {}
    
    state_dict['name'] = a.string.strip()
    
    groups = re.search('\d\din(?P<id>\d\d)(?P<abbrev>\w\w)', a.attrs['href'])
    state_dict.update(groups.groupdict())
    
    state_refs.append(state_dict)
    
df_state_refs = pd.DataFrame(state_refs).sort_values('id').reset_index(drop=True)

display(df_state_refs)

,abbrev,id,name
0,al,01,Alabama
1,ak,02,Alaska
2,az,03,Arizona
3,ar,04,Arkansas
4,ca,05,California
5,co,06,Colorado
6,ct,07,Connecticut
7,de,08,Delaware
8,dc,09,District of Columbia
9,fl,10,Florida


In [152]:
from requests import Session
from urllib.parse import urljoin

class LiveServerSession(Session):
    def __init__(self, prefix_url=None, *args, **kwargs):
        super(LiveServerSession, self).__init__(*args, **kwargs)
        self.prefix_url = prefix_url

    def request(self, method, url, *args, **kwargs):
        url = urljoin(self.prefix_url, url)
        return super(LiveServerSession, self).request(method, url, *args, **kwargs)

def lookup_state(key, val):
    df = df_state_refs
    condition = df[key] == val
    state_info = df[condition].to_dict(orient='records')[0]
    return state_info

def get_soi_data(lookup, year):
    state_info = lookup_state(*lookup)
    filename = "{}in{}{}".format(
        str(year)[2:],
        state_info['id'],
        state_info['abbrev']
    )
    pattern = r'{}\.(xlsx?|zip)'.format(filename)
    url = soup.find('a', href=re.compile(pattern)).attrs['href']
    baseUrl = 'https://www.irs.gov'
        
    try:
        with LiveServerSession(baseUrl) as s:
            r = s.get(url)
        assert r.status_code == 200
    except:
        raise
        
    return r

def get_soi_df(lookup, year):
    '''
    get cumulative data as df given lookup and year
    '''
    from io import BytesIO
    from zipfile import ZipFile
    import itertools
    
    r = get_soi_data(lookup, year)
    
    print(r.url)
    
    if ((lookup, year) in CACHED_DFS):
        return CACHED_DFS[(lookup, year)]
    
    pd_options = {
        'header': None
    }
    
    if '.xls' in r.url:
        with BytesIO(r.content) as fh:
            df = pd.read_excel(fh, **pd_options)
    
    elif '.zip' in r.url:
        with ZipFile(BytesIO(r.content)) as my_zipfile:
            for file in my_zipfile.namelist():
                with my_zipfile.open(file) as fh:
                    df = pd.read_excel(fh, **pd_options)
    
    CACHED_DFS[(lookup, year)] = df
    
    return df

In [286]:
def get_indices(df):
    from collections import defaultdict

    iterator = df.iterrows()
    indices = defaultdict(lambda: None)

    while not np.all([indices[k] for k in ['place', 'amt']]):
        idx, row = next(iterator)

        if row.str.contains('alabama', flags=re.IGNORECASE).any() and not indices['place']:
            indices['place'] = idx

        if row.apply(str).str.contains('100,?000').any() and not indices['amt']:
            indices['amt'] = idx

    return dict(indices)

def format_soi_df(df):
    import numpy as np

    my_df = df.copy()
        
    # identity target rows for indexing
    indices = get_indices(my_df)

    # fill null holes in columns for multiindexing
    my_df.loc[indices['place']].fillna(method='ffill', inplace=True)
    my_df.loc[indices['place']].fillna('', inplace=True)
    my_df.loc[indices['place']] = my_df.loc[indices['place']].str.strip()
    my_df.loc[indices['amt']].fillna('All returns', inplace=True)
    
    def format_column(x):
        if isinstance(x, float) and np.isnan(x):
            return x
        elif isinstance(x, str) and 'Under' in x:
            x = '0'
        elif isinstance(x, str) and 'Breakeven' in x:
            x = '0'
        elif isinstance(x, str) and 'All' in x:
            x = '-1'
        else:
            x = str(x)

        try:
            return float(x.split()[0].replace('$', '').replace(',', ''))
        except:
            return x.split()[0]

    my_df.loc[indices['amt']] = my_df.loc[indices['amt']].apply(format_column)

    # drop rows with any null values in arbitrary range – we don't need these anymore
    my_df = my_df[~my_df.iloc[:, 1:4].isnull().any(axis=1)]
    
    # get rid of footnotes in data
    my_df.replace('\s*\[\d+\]', '', regex=True, inplace=True)

    # set indices
    my_df = my_df.transpose().set_index([*indices.values()])
    my_df.index.set_names([*indices.keys()], inplace=True)

    # set columns
    my_df.columns = my_df.iloc[0]
    my_df = my_df.iloc[1:]
    my_df.columns.rename('Item', inplace=True)
    
    my_df = my_df[my_df.iloc[:, 3] != my_df.columns[3]] # drop rows that match header

    return my_df

In [247]:
def realign_tables(df):
    
    
    new_starts = df[df.iloc[:, 0].str.contains('^Table 2.', flags=re.IGNORECASE, na=False)].index
    
    if not new_starts.any():
        new_starts = df[df.iloc[:, 0].str.contains('^Tax Year 200', flags=re.IGNORECASE, na=False)].index
    
    new_starts = new_starts.drop(0)
    
    if(new_starts.any()):
        dfs = []
        iterator = iter(new_starts.values)
        
        curr = 0
        
#         dfs.append(df.loc[:(new_starts.values[0] - 1), 0])
        
        for next_val in new_starts.values:
            start = curr
            curr = next_val
            
            dfs.append(df.loc[start:(curr-1), :].reset_index(drop=True))
        
        dfs.append(df.loc[curr:, :].reset_index(drop=True))
        
        concat = pd.concat(dfs, axis=1, ignore_index=True)
        
        # begone null columns
        concat = concat.loc[:, ~concat.isnull().all()]
        
        return concat
        
    else:
        
        return df


In [289]:
import os

for year in range(2005, 1999, -1):
    print(datetime.datetime.now(), year, 'Fetching data')
    df = get_soi_df(('abbrev', 'cm'), year)
    print(datetime.datetime.now(), year, 'Realigning data')
    df = realign_tables(df)
    
#     display(df.head(n=20))
    
#     df.to_csv('sample.csv')
    
    print(datetime.datetime.now(), year, 'Formatting data')
    
    if os.path.isfile(f'./working/{year}.csv'):
        df = pd.read_csv(f'./working/{year}.csv', index_col=0, header=0)
    
    my_df = format_soi_df(df)
#     print()
    display(my_df.head(n=10))

2019-03-25 18:54:45.552109 2005 Fetching data
https://www.irs.gov/pub/irs-soi/05in54cm.xls
2019-03-25 18:54:45.576384 2005 Realigning data
2019-03-25 18:54:45.585121 2005 Formatting data


Item                                                                   NaN     \
place         amt                                                               
UNITED STATES -1.0                                                          1   
               0.0                                                          2   
               50000.0                                          Size of...  3   
               75000.0                                                 NaN  4   
               100000.0                                                     5   
               200000.0                                                     6   
ALABAMA       -1.0                                                          1   
               0.0                                                          2   
               50000.0                               Size of adjusted g...  3   
               75000.0                                                 NaN  4   

Item                    Number of returns Number of joint returns  \
place         amt                                                   
UNITED STATES -1.0              135257620                52607676   
               0.0               92150166                20743943   
               50000.0           18221115                11329459   
               75000.0           10499106                 8296546   
               100000.0          10797979                 9193700   
               200000.0           3589254                 3044028   
ALABAMA       -1.0                1955914                  774753   
               0.0                1428663                  348843   
               50000.0             240191                  172740   
               75000.0             134090                  115859   

Item                    Number with paid preparer's signature  \
place         amt                                               
UNITED STATES -1.0                                   80455243   
               0.0                                   53622647   
               50000.0                               11025624   
               75000.0                                6260725   
               100000.0                               6678965   
               200000.0                               2867282   
ALABAMA       -1.0                                    1272190   
               0.0                                     930445   
               50000.0                                 155834   
               75000.0                                  82891   

Item                    Number of exemptions Adjusted gross income (AGI)  \
place         amt                                                          
UNITED STATES -1.0                 273738434                  7364640131   
               0.0                 159649737                  1797097083   
               50000.0              44189517                  1119634632   
               75000.0              28555195                   905336768   
               100000.0             30919226                  1429575727   
               200000.0             10424759                  2112995921   
ALABAMA       -1.0                   4093805                    88628735   
               0.0                   2670264                    27817732   
               50000.0                608134                    14753559   
               75000.0                374372                    11540510   

Item                    Salaries and wages in AGI: Number      Amount  \
place         amt                                                       
UNITED STATES -1.0                              114060887  5161583318   
               0.0                               75422766  1541276272   
               50000.0                           16299827   896339313   
               75000.0                            9520214   721137490   
               100000.0                           9782173  1083175

2019-03-25 18:54:45.691183 2004 Fetching data
https://www.irs.gov/pub/irs-soi/04in54cm.xls
2019-03-25 18:54:45.716889 2004 Realigning data
2019-03-25 18:54:45.725197 2004 Formatting data


Item                                                                   NaN     \
place         amt                                                               
UNITED STATES -1.0                                                          1   
               0.0                                                          2   
               50000.0                                          Size of...  3   
               75000.0                                                 NaN  4   
               100000.0                                                     5   
               200000.0                                                     6   
ALABAMA       -1.0                                                          1   
               0.0                                                          2   
               50000.0                               Size of adjusted g...  3   
               75000.0                                                 NaN  4   

Item                    Number of returns Number of joint returns  \
place         amt                                                   
UNITED STATES -1.0              133092565                52225721   
               0.0               92277037                21593939   
               50000.0           17922902                11513644   
               75000.0           10015637                 8063363   
               100000.0           9815321                 8438738   
               200000.0           3061668                 2616037   
ALABAMA       -1.0                1910403                  768809   
               0.0                1416612                  364035   
               50000.0             234243                  173791   
               75000.0             125827                  110195   

Item                    Number with paid preparer's signature  \
place         amt                                               
UNITED STATES -1.0                                   78125181   
               0.0                                   53216912   
               50000.0                               10700352   
               75000.0                                5872471   
               100000.0                               5946200   
               200000.0                               2389246   
ALABAMA       -1.0                                    1224390   
               0.0                                     909501   
               50000.0                                 150198   
               75000.0                                  76742   

Item                    Number of exemptions Adjusted gross income (AGI)  \
place         amt                                                          
UNITED STATES -1.0                 269662776                  6745101602   
               0.0                 160217108                  1788706363   
               50000.0              44303111                  1100740167   
               75000.0              27732581                   862762850   
               100000.0             28429329                  1294760029   
               200000.0              8980647                  1698132194   
ALABAMA       -1.0                   4006258                    80884836   
               0.0                   2658350                    27332223   
               50000.0                603145                    14380699   
               75000.0                355931                    10814201   

Item                    Salaries and wages in AGI: Number      Amount  \
place         amt                                                       
UNITED STATES -1.0                              112534497  4917551535   
               0.0                               75627439  1533846536   
               50000.0                           16166931   891303503   
               75000.0                            9169619   699009194   
               100000.0                           8962650   999027

2019-03-25 18:54:45.833942 2003 Fetching data
https://www.irs.gov/pub/irs-soi/03in54cm.xls
2019-03-25 18:54:45.856234 2003 Realigning data
2019-03-25 18:54:45.888276 2003 Formatting data


...

Item                                               NaN    Number of returns  \
place         amt                                                             
UNITED STATES -1.0                                      1         130836098   
               0.0                                      2          50248794   
               20000.0                                  3          18650546   
               30000.0                             NaN  4          24319476   
               50000.0   Size of adjusted gross income  5          17628351   
               75000.0                             NaN  6           9129894   
               100000.0                                 7           8391951   
               200000.0                                 8           2467086   
ALABAMA       -1.0                                      1           1882572   
               0.0                                      2            816584   

Item                    Number of joint returns  \
place         amt                                 
UNITED STATES -1.0                     51477158   
               0.0                      7387934   
               20000.0                  5090957   
               30000.0                 10095791   
               50000.0                 11977409   
               75000.0                  7511701   
               100000.0                 7285992   
               200000.0                 2127374   
ALABAMA       -1.0                       766190   
               0.0                       132892   

Item                    Number with paid preparer's signature  \
place         amt                                               
UNITED STATES -1.0                                   74832886   
               0.0                                   27881024   
               20000.0                               10746359   
               30000.0                               13864118   
               50000.0                               10272065   
               75000.0                                5203512   
               100000.0                               4962840   
               200000.0                               1902968   
ALABAMA       -1.0                                    1176133   
               0.0                                     504101   

Item                    Number of exemptions Adjusted gross income (AGI)  \
place         amt                                                          
UNITED STATES -1.0                 263875135                  6015047033   
               0.0                  73922834                   385367487   
               20000.0              36043469                   461857601   
               30000.0              51514627                   950261330   
               50000.0              44818405                  1081036334   
               75000.0              25737602                   784946802   
               100000.0             24515756                  1103520609   
               200000.0              7322442                  1248056881   
ALABAMA       -1.0                   3938787                    72426176   
               0.0                   1357965                     7373588   

Item                    Salaries and wages in AGI:  Number      Amount  \
place         amt                                                        
UNITED STATES -1.0                               111008171  4562427437   
               0.0                                38751335   373560036   
               20000.0                            16246786   376820992   
               30000.0                            21797281   785529009   
               50000.0                            16000889   881900609   
               75000.0                             8400593   642597210   
               100000.0                            7681868   857977162   
               200000.0                            2129419   644042424   
ALAB

2019-03-25 18:54:46.248549 2001 Fetching data
https://www.irs.gov/pub/irs-soi/01in54cm.xls
2019-03-25 18:54:46.283633 2001 Realigning data
2019-03-25 18:54:46.348346 2001 Formatting data


Item                         NaN      NaN Returns Count Joint Returns Count  \
place         amt                                                             
United States -1.00       Total   Returns     130977219            51193206   
               0.00          and     Loss       1752721              516701   
               0.01        Under    10000      25148294             1929894   
               10000.00    Under    20000      23493788             4606740   
               20000.00    Under    30000      18741155             5064275   
               30000.00    Under    50000      24343572            10240186   
               50000.00    Under    75000      17639483            12083689   
               75000.00    Under   100000       8943249             7349674   
               100000.00   Under   150000       6372267             5491853   
               150000.00   Under   200000       1966655             1702026   

Item                     Single Returns Count Head of Household Count  \
place         amt                                                       
United States -1.00                  58376029                18598611   
               0.00                    997706                  114269   
               0.01                  19376052                 3408644   
               10000.00              12736825                 5646151   
               20000.00               8949946                 4161891   
               30000.00               9893521                 3510065   
               50000.00               4034341                 1225130   
               75000.00               1205447                  298626   
               100000.00               687723                  143756   
               150000.00               208036                   40182   

Item                     Number of Farm Returns Paid Preparer Returns Count  \
place         amt                                                             
United States -1.00                     2048843                    73973374   
               0.00                      126134                     1132623   
               0.01                      193078                    12506368   
               10000.00                  261883                    13762391   
               20000.00                  250445                    10617934   
               30000.00                  430112                    13741400   
               50000.00                  377334                    10228017   
               75000.00                  177581                     5077444   
               100000.00                 119318                     3670706   
               150000.00                  40979                     1254165   

Item                          Number of Returns      Amount     ...      \
place         amt                                               ...       
United States -1.00                   111521029  4555746889     ...       
               0.00                      443166    11347668     ...       
               0.01                    20093979   101024299     ...       
               10000.00                18975517   265221497     ...       
               20000.00                16336925   378995122     ...       
               30000.00                21781241   781856685     ...       
               50000.00                15942274   871919105     ...       
               75000.00                 8176707   618192064     ...       
               100000.00                5807557   591114972     ...       
               150000.00                1755012   241136772     ...       

Item                          Number of Returns      Amount  \
place         amt                                             
United States -1.00                    37379429   199410093   
               0.00                      176422     1152591   
               0.01                      498342      957213   
         

2019-03-25 18:54:46.620346 2000 Fetching data
https://www.irs.gov/pub/irs-soi/00in54cm.xls
2019-03-25 18:54:46.653848 2000 Realigning data
2019-03-25 18:54:46.710350 2000 Formatting data


Item                   NaN      NaN Returns Count Dependent Exemptions Count  \
place   amt                                                                    
Alabama -1.00       Total   Returns       1904150                    1366454   
         0.00          and     Loss         13320                       4871   
         0.01        Under    10000        415090                     178778   
         10000.00    Under    20000        423565                     314916   
         20000.00    Under    30000        286275                     210288   
         30000.00    Under    50000        331729                     253883   
         50000.00    Under    75000        228193                     201719   
         75000.00    Under   100000        102628                      99754   
         100000.00   Under   150000         62364                      60887   
         150000.00   Under   200000         17429                      17109   

Item               Joint Returns Count Single Returns Count  \
place   amt                                                   
Alabama -1.00                   772772               705643   
         0.00                     5888                 6298   
         0.01                    38420               277191   
         10000.00                87935               172825   
         20000.00                92563               107537   
         30000.00               179897                95104   
         50000.00               183234                30663   
         75000.00                91826                 7997   
         100000.00               56546                 4505   
         150000.00               15627                 1408   

Item               Head of Household Count Number of Farm Returns  \
place   amt                                                         
Alabama -1.00                       393607                  51896   
         0.00                          550                   2097   
         0.01                        94147                   4559   
         10000.00                   154939                   6701   
         20000.00                    78640                   6305   
         30000.00                    49412                  11121   
         50000.00                    12060                  10220   
         75000.00                     2245                   5016   
         100000.00                     979                   3020   
         150000.00                     280                   1038   

Item               Paid Preparer Returns Count      Number of Returns  \
place   amt                                                             
Alabama -1.00                          1125294                1654590   
         0.00                            10381                   4232   
         0.01                           215780                 345606   
         10000.00                       257889                 359949   
         20000.00                       168833                 254341   
         30000.00                       201511                 299445   
         50000.00                       140640                 207254   
         75000.00                        60657                  93693   
         100000.00                       37562                  55753   
         150000.00                       12152                  14904   

Item                   ...          Number of Returns      Amount  \
place   amt            ...                                          
Alabama -1.00          ...                     522970     1592985   
         0.00          ...                       2060        7581   
         0.01          ...                       6286        8061   
         10000.00      ...                      26336       18224   
         20000.00      ...                      51230       48998   
         30000.00      ...                     125489      187481   
         50000.00      .

In [285]:
# my_df[my_df['Returns Count'] != 'Returns Count']
# my_df.columns[3]


Item                          NaN      NaN Returns Count  \
place         amt                                          
Alabama       -1.00        Total   Returns       1904150   
               0.00           and     Loss         13320   
               0.01         Under    10000        415090   
               10000.00     Under    20000        423565   
               20000.00     Under    30000        286275   
               30000.00     Under    50000        331729   
               50000.00     Under    75000        228193   
               75000.00     Under   100000        102628   
               100000.00    Under   150000         62364   
               150000.00    Under   200000         17429   
               200000.00    Under   500000         18357   
               500000.00    Under  1000000          3636   
               1000000.00     and     Over          1564   
Alaska        -1.00        Total   Returns        328747   
               0.00           and     Loss          2481   
               0.01         Under    10000         87508   
               10000.00     Under    20000         47394   
               20000.00     Under    30000         38554   
               30000.00     Under    50000         55022   
               50000.00     Under    75000         44700   
               75000.00     Under   100000         25647   
               100000.00    Under   150000         18261   
               150000.00    Under   200000          4687   
               200000.00    Under   500000          3708   
               500000.00    Under  1000000           567   
               1000000.00     and     Over           218   
Arizona       -1.00        Total   Returns       2152615   
               0.00           and     Loss         18190   
               0.01         Under    10000        374530   
               10000.00     Under    20000        422571   
...                           ...      ...           ...   
Wyoming        150000.00    Under   200000          2088   
               200000.00    Under   500000          2531   
               500000.00    Under  1000000           568   
               1000000.00     and     Over           473   
Other Areas   -1.00        Total   Returns       1474174   
               0.00           and     Loss        454996   
               0.01         Under    10000        395586   
               10000.00     Under    20000        208848   
               20000.00     Under    30000        112358   
               30000.00     Under    50000        115373   
               50000.00     Under    75000         68667   
               75000.00     Under   100000         34065   
               100000.00    Under   150000         33287   
               150000.00    Under   200000         16021   
               200000.00    Under   500000         24393   
               500000.00    Under  1000000          6290   
               1000000.00     and     Over          4290   
United States -1.00        Total   Returns     130122204   
               0.00           and     Loss       1464336   
               0.01         Under    10000      25947174   
               10000.00     Under    20000      23678120   
               20000.00     Under    30000      18533555   
               30000.00     Under    50000      23878431   
               50000.00     Under    75000      17263552   
               75000.00     Under   100000       8547241   
               100000.00    Under   150000       6098349   
               150000.00    Under   200000       1947393   
               200000.00    Under   500000       2120925   
               500000.00    Under  1000000        402060   
               1000000.00     and     Over        241068   

Item                      Dependent Exemptions Count Joint Returns Count  \
place         amt                                                          
Alabama       -1.00                          1366454              7

In [144]:
import datetime

RANGES = {
    'Under $50k': (0, 5e4),     #    0 <= ... <  50k
    '$50-$100k': (5e4, 1e5),   #  50k <= ... < 100k
    '$100-$200k': (1e5, 2e5),  # 100k <= ... < 200k
    '$200k and greater': (2e5,)    # 200k <= ...
}

DIMENSIONS = [
    'Number of returns',
    'Adjusted gross income (AGI)',
]

def build_query(my_place, lte, gt=None):
    my_query = ''
    
    my_query += f'{lte} <= amt'
    
    if (gt):
        my_query += f' < {gt} '
    
    my_query += f' and place == "{my_place}"'
    
    return my_query

data = []

def format_place(x):
    df = df_state_refs[df_state_refs.name.str.contains(x, case=False)]
    return df.abbrev.iloc[0]

for year in range(2016, 2004, -1):
    print(datetime.datetime.now(), year, 'Fetching data')
    df = get_soi_df(('abbrev', 'cm'), year)
    print(datetime.datetime.now(), year, 'Realigning data')
    df = realign_tables(df)
    print(datetime.datetime.now(), year, 'Formatting data')
    my_df = format_soi_df(df)
    print()
    
    for place in df_state_refs.name:
        my_place = place.upper()
        
        for range_name, my_range in RANGES.items():
            my_query = build_query(my_place, *my_range)
#             my_value = my_df.query(my_query)['Number of returns'].sum()
            for my_dimension in DIMENSIONS:
                my_value = my_df.query(my_query)[my_dimension].sum()

                data.append((
                    my_dimension,
                    f'1/1/{year}',
                    format_place(my_place),
                    range_name,
                    my_value
                ))
        
df = pd.DataFrame(data)

2019-03-25 17:12:24.163891 2016 Fetching data
2019-03-25 17:12:24.164969 2016 Realigning data
2019-03-25 17:12:24.166643 2016 Formatting data

2019-03-25 17:12:27.219924 2015 Fetching data
2019-03-25 17:12:27.220027 2015 Realigning data
2019-03-25 17:12:27.220725 2015 Formatting data

2019-03-25 17:12:29.985256 2014 Fetching data
2019-03-25 17:12:29.985361 2014 Realigning data
2019-03-25 17:12:29.986052 2014 Formatting data

2019-03-25 17:12:32.655898 2013 Fetching data
2019-03-25 17:12:32.656002 2013 Realigning data
2019-03-25 17:12:32.656682 2013 Formatting data

2019-03-25 17:12:35.167298 2012 Fetching data
2019-03-25 17:12:35.167407 2012 Realigning data
2019-03-25 17:12:35.168090 2012 Formatting data

2019-03-25 17:12:37.782952 2011 Fetching data
2019-03-25 17:12:37.783055 2011 Realigning data
2019-03-25 17:12:37.783741 2011 Formatting data

2019-03-25 17:12:40.294166 2010 Fetching data
2019-03-25 17:12:40.294279 2010 Realigning data
2019-03-25 17:12:40.307614 2010 Formatting data


In [145]:
df.columns = ('dimension', 'date', 'group', 'range', 'val')

pivot = pd.pivot_table(df, values='val', columns='range', index=['dimension', 'group', 'date'])

pivot = pivot[['Under $50k', '$50-$100k', '$100-$200k', '$200k and greater']]

In [138]:
subset = pivot.xs('1/1/2007', level=1, drop_level=False)
subset[:] = -1
pivot.update(subset)
pivot.query('group not in ["cm", "oa"]').query('date in ["1/1/2006", "1/1/2016"]').to_csv('sample.csv')

/Users/pjudge/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/pjudge/Library/Python/3.7/lib/python/site-packages/pandas/core/frame.py:3123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer(key, value)
/Users/pjudge/Library/Python/3.7/lib/python/site-packages/pandas/core/frame.py:3111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [146]:
pivot.query('group in ["il"]')

range                                       Under $50k    $50-$100k  \
dimension                   group date                                
Adjusted gross income (AGI) il    1/1/2005  75252964.0   93382961.0   
                                  1/1/2006  75009056.0   96364475.0   
                                  1/1/2007  76096097.0   99868100.0   
                                  1/1/2008  73365620.0   98937338.0   
                                  1/1/2009  77801110.0   95598997.0   
                                  1/1/2010  72224644.0   96752757.0   
                                  1/1/2011  70649498.0   96883967.0   
                                  1/1/2012  70008538.0   97152886.0   
                                  1/1/2013  69667300.0   98076995.0   
                                  1/1/2014  69498881.0   99289262.0   
                                  1/1/2015  69623626.0  100296857.0   
                                  1/1/2016  69069645.0  100072315.0   
Number of returns           il    1/1/2005   3832554.0    1319905.0   
                                  1/1/2006   3855445.0    1357931.0   
                                  1/1/2007   4304768.0    1404540.0   
                                  1/1/2008   3865517.0    1390269.0   
                                  1/1/2009   3849417.0    1345143.0   
                                  1/1/2010   3834120.0    1358185.0   
                                  1/1/2011   3860445.0    1358033.0   
                                  1/1/2012   3754050.0    1359260.0   
                                  1/1/2013   3721230.0    1370470.0   
                                  1/1/2014   3671840.0    1386000.0   
                                  1/1/2015   3638660.0    1399680.0   
                                  1/1/2016   3569220.0    1397680.0   

range                                        $100-$200k  $200k and greater  
dimension                   group date                                      
Adjusted gross income (AGI) il    1/1/2005   67811056.0         98874476.0  
                                  1/1/2006   75895156.0        114966917.0  
                                  1/1/2007   84171574.0        132529144.0  
                                  1/1/2008   86444393.0        115801241.0  
                                  1/1/2009   83605198.0         93433098.0  
                                  1/1/2010   86919503.0        106365303.0  
                                  1/1/2011   91664666.0        112216060.0  
                                  1/1/2012   96566785.0        139391854.0  
                                  1/1/2013  101512794.0        129944416.0  
                                  1/1/2014  106932210.0        148468300.0  
                                  1/1/2015  111530789.0        160190360.0  
                                  1/1/2016  112374651.0        155405619.0  
Number of returns           il    1/1/2005     513157.0           170577.0  
                                  1/1/2006     572474.0           193844.0  
                                  1/1/2007     632996.0           217054.0  
                                  1/1/2008     649579.0           207061.0  
                                  1/1/2009     627582.0           186041.0  
                                  1/1/2010     651227.0           200337.0  
                                  1/1/2011     683830.0           219720.0  
                                  1/1/2012     717840.0           245950.0  
                                  1/1/2013     752490.0           256510.0  
                                  1/1/2014     790340.0           282930.0  
                                  1/1/2015     822510.0           301130.0  
                                  1/1/2016     827510.0           305690.0